# Divide and Conquer Enumerative Approach and SMT Based Verification Oracle

The Divide and Conquer Enumerative Approach accepts a CFG written in z3 expressions and synthesizes z3 expressions, and the SMT Based Verification Oracle operates on z3 expressions.

In [1]:
from z3 import *

from iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation import iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation


z3_S = Int('S')
z3_B = Bool('B')
z3_x = Int('x')
z3_y = Int('y')

z3_non_terminals = { z3_S, z3_B }
z3_terminals = { z3_x, z3_y }
# non_terminals to a set of tuples containing the production_rule in reverse_polish_notation
z3_production_rules = {
  z3_S: {
    # x
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(z3_x)),
    # y
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(z3_y)),
    # 0
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(IntVal(0))),
    # 1
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(IntVal(1))),
    # + S S
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(z3_S + z3_S)),
    # - S S
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(z3_S - z3_S)),
    # ite B S S
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(If(z3_B, z3_S, z3_S))),
  },
  z3_B: {
    # and B B
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(And(z3_B, z3_B))),
    # or B B
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(Or(z3_B, z3_B))),
    # not B
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(Not(z3_B))),
    # <= S S
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(z3_S <= z3_S)),
    # = S S
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(z3_S == z3_S)),
    # >= S S
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(z3_S >= z3_S)),
  }
}
z3_start_symbol = z3_S


z3_input_variable_list = [z3_x, z3_y]
z3_function_declaration = max2 = Function('max2', IntSort(), IntSort(), IntSort())
z3_constraint = And(max2(z3_x, z3_y) >= z3_x, max2(z3_x, z3_y) >= z3_y, Or(max2(z3_y, z3_x) == z3_x, max2(z3_y, z3_x) == z3_y))

## Initializing `z3_verification_oracle_instance`

In [2]:
from z3_verification_oracle import z3_verification_oracle


z3_verification_oracle_instance = z3_verification_oracle(z3_input_variable_list, z3_function_declaration, z3_constraint)
next(z3_verification_oracle_instance)

## Initializing `z3_divide_and_conquer_enumerative_approach_instance`

In [3]:
from z3_divide_and_conquer_enumerative_approach import z3_divide_and_conquer_enumerative_approach

z3_divide_and_conquer_enumerative_approach_instance = z3_divide_and_conquer_enumerative_approach(
    z3_non_terminals,
    z3_terminals,
    z3_production_rules,
    z3_start_symbol,
    z3_B,
    z3_function_declaration,
    z3_constraint
)

## Program Synthesis and Verification

In [4]:
z3_expr_ref = next(z3_divide_and_conquer_enumerative_approach_instance)

 counterexample_set: set()
 term_to_covered_counterexample_set_dict: {x: set(), 0: set(), 1: set(), y: set()}
 predicate_to_covered_counterexample_set_dict: {Not(1 >= y): set(), Not(x >= 0): set(), Not(x == y): set(), Not(x >= 1): set(), y <= 1: set(), y >= 0: set(), 1 == y: set(), 1 == x: set(), 1 <= y: set(), x >= 1: set(), 1 <= x: set(), y >= x: set(), Not(1 == x): set(), Not(y == x): set(), y == 0: set(), Not(1 == y): set(), Not(x == 0): set(), 0 >= y: set(), Not(y >= 0): set(), Not(x == 1): set(), 0 >= x: set(), Not(y >= 1): set(), y <= 0: set(), x == 1: set(), y == x: set(), Not(0 >= x): set(), x <= 1: set(), x >= 0: set(), Not(0 >= y): set(), y <= x: set(), 0 == y: set(), 0 == x: set(), x >= y: set(), 0 <= y: set(), Not(x >= y): set(), 0 <= x: set(), Not(y == 0): set(), y >= 1: set(), Not(y == 1): set(), x == 0: set(), Not(0 == x): set(), Not(1 >= x): set(), x <= 0: set(), x == y: set(), 1 >= y: set(), Not(0 == y): set(), 1 >= x: set(), x <= y: set(), Not(y >= x): set(), y == 1:

In [5]:
print(f'z3_expr_ref {z3_expr_ref}')

z3_counterexample_input = z3_verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    z3_expr_ref = z3_divide_and_conquer_enumerative_approach_instance.send(z3_counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

z3_expr_ref x
z3_counterexample_input frozendict.frozendict({x: -1, y: 0})


 counterexample: frozendict.frozendict({x: -1, y: 0})
 counterexample_set: {frozendict.frozendict({x: -1, y: 0})}
 term_to_covered_counterexample_set_dict: {x: set(), 0: {frozendict.frozendict({x: -1, y: 0})}, 1: set(), y: {frozendict.frozendict({x: -1, y: 0})}}
 predicate_to_covered_counterexample_set_dict: {Not(1 >= y): set(), Not(x >= 0): {frozendict.frozendict({x: -1, y: 0})}, Not(x == y): {frozendict.frozendict({x: -1, y: 0})}, Not(x >= 1): {frozendict.frozendict({x: -1, y: 0})}, y <= 1: {frozendict.frozendict({x: -1, y: 0})}, y >= 0: {frozendict.frozendict({x: -1, y: 0})}, 1 == y: set(), 1 == x: set(), 1 <= y: set(), x >= 1: set(), 1 <= x: set(), y >= x: {frozendict.frozendict({x: -1, y: 0})}, Not(1 == x): {frozendict.frozendict({x: -1, y: 0})}, Not(y == x): {frozendict.frozendict({x: -1, y: 0})}, y == 0: {frozendict.frozendict({x: -1, y: 0})}, Not(1 == y): {frozendict.frozendict({x: -1, y: 0})}, Not(x == 0): {frozendict.frozendict({x: -1, y: 0})}, 0 >= y: {frozendict.frozendict(

In [6]:
print(f'z3_expr_ref {z3_expr_ref}')

z3_counterexample_input = z3_verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    z3_expr_ref = z3_divide_and_conquer_enumerative_approach_instance.send(z3_counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

 counterexample: frozendict.frozendict({x: -1, y: -1})
 counterexample_set: {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}
 term_to_covered_counterexample_set_dict: {x: {frozendict.frozendict({x: -1, y: -1})}, 0: {frozendict.frozendict({x: -1, y: 0})}, 1: set(), y: {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}}
 predicate_to_covered_counterexample_set_dict: {Not(1 >= y): set(), Not(x >= 0): {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, Not(x == y): {frozendict.frozendict({x: -1, y: 0})}, Not(x >= 1): {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, y <= 1: {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, y >= 0: {frozendict.frozendict({x: -1, y: 0})}, 1 == y: set(), 1 == x: set(), 1 <= y: set(), x >= 1: set(), 1 <= x: set(), y >= x: {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, Not(1 == x): {frozendi

z3_expr_ref 0
z3_counterexample_input frozendict.frozendict({x: -1, y: -1})


{1 >= x, Not(1 >= y), Not(x >= 0), Not(x == y), Not(x >= 1), y <= 1, y >= 0, 1 == y, 1 == x, 1 <= y, x >= 1, 1 <= x, y >= x, y == 1, Not(1 == x), Not(y == x), y == 0, Not(1 == y), Not(x == 0), 0 >= y, Not(y >= 0), Not(x == 1), 0 >= x, Not(y >= 1), y <= 0, x == 1, y == x, Not(0 >= x), x <= 1, x >= 0, Not(0 >= y), y <= x, 0 == y, 0 == x, x >= y, 0 <= y, Not(x >= y), 0 <= x, Not(y == 0), y >= 1, Not(y == 1), x == 0, Not(0 == x), x <= 0, x == y, 1 >= y, Not(0 == y), Not(1 >= x), Not(y >= x), x <= y} {Not(1 >= y): set(), Not(x >= 0): {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, Not(x == y): {frozendict.frozendict({x: -1, y: 0})}, Not(x >= 1): {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, y <= 1: {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, y >= 0: {frozendict.frozendict({x: -1, y: 0})}, 1 == y: set(), 1 == x: set(), 1 <= y: set(), x >= 1: set(), 1 <= x: set(), y >= x: {frozendict.frozendict(

In [7]:
print(f'z3_expr_ref {z3_expr_ref}')

z3_counterexample_input = z3_verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    z3_expr_ref = z3_divide_and_conquer_enumerative_approach_instance.send(z3_counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

z3_expr_ref y


 counterexample: frozendict.frozendict({x: 0, y: -1})
 counterexample_set: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}
 term_to_covered_counterexample_set_dict: {x: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: -1})}, 0: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0})}, 1: set(), y: {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}}
 predicate_to_covered_counterexample_set_dict: {Not(1 >= y): set(), Not(x >= 0): {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, Not(x == y): {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0})}, Not(x >= 1): {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, y <= 1: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, y >= 

z3_counterexample_input frozendict.frozendict({x: 0, y: -1})


     left_subtree_counterexample_set: {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}
     left_subtree_term_to_covered_counterexample_set_dict: {x: {frozendict.frozendict({x: -1, y: -1})}, 0: {frozendict.frozendict({x: -1, y: 0})}, 1: set(), y: {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}}
     left_subtree_predicate_set: {Not(x == y), x >= 0, y <= x, 0 == y, y >= 0, 0 == x, x >= y, 0 <= y, Not(x >= y), 0 <= x, Not(y == 0), y >= x, x == 0, Not(y == x), Not(0 == x), y == 0, Not(y >= x), Not(x == 0), Not(y >= 0), x == y, Not(0 == y), y == x, x <= y}
     left_subtree_predicate_to_covered_counterexample_set_dict: {Not(x == y): {frozendict.frozendict({x: -1, y: 0})}, x >= 0: set(), y <= x: {frozendict.frozendict({x: -1, y: -1})}, 0 == y: {frozendict.frozendict({x: -1, y: 0})}, y >= 0: {frozendict.frozendict({x: -1, y: 0})}, 0 == x: set(), x >= y: {frozendict.frozendict({x: -1, y: -1})}, 0 <= y: {frozendict.frozendict({x: -1, y

In [8]:
print(f'z3_expr_ref {z3_expr_ref}')

z3_counterexample_input = z3_verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    z3_expr_ref = z3_divide_and_conquer_enumerative_approach_instance.send(z3_counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

z3_expr_ref If(Not(x >= 0), y, x)


 counterexample: frozendict.frozendict({x: -1, y: -2})
 counterexample_set: {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}
 term_to_covered_counterexample_set_dict: {x: {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: -1})}, 0: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0})}, 1: set(), y: {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}}
 predicate_to_covered_counterexample_set_dict: {Not(1 >= y): set(), Not(x >= 0): {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, Not(x == y): {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0})}, Not(x >= 1): {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1}), frozendict.f

z3_counterexample_input frozendict.frozendict({x: -1, y: -2})


{Not(x >= 0): {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: -1, y: -1})}, Not(x == y): {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1})}, x >= 0: {frozendict.frozendict({x: 0, y: -1})}, 0 == y: set(), y >= 0: set(), 0 == x: {frozendict.frozendict({x: 0, y: -1})}, x >= y: {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: -1})}, 0 <= y: set(), Not(x >= y): set(), 0 <= x: {frozendict.frozendict({x: 0, y: -1})}, Not(y == 0): {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: -1})}, y >= x: {frozendict.frozendict({x: -1, y: -1})}, x == 0: {frozendict.frozendict({x: 0, y: -1})}, Not(y == x): {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1})}, Not(0 == x): {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: -1, y: -1})}, y == 0: set(), Not(y >= x): {frozendict.frozendict({x: -1, y: -2})

In [9]:
print(f'z3_expr_ref {z3_expr_ref}')

z3_counterexample_input = z3_verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    z3_expr_ref = z3_divide_and_conquer_enumerative_approach_instance.send(z3_counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

z3_expr_ref If(y <= x, x, 0)


 counterexample: frozendict.frozendict({x: 1, y: 2})
 counterexample_set: {frozendict.frozendict({x: -1, y: -1}), frozendict.frozendict({x: 1, y: 2}), frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0})}
 term_to_covered_counterexample_set_dict: {x: {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: -1})}, 0: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0})}, 1: set(), y: {frozendict.frozendict({x: 1, y: 2}), frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}}
 predicate_to_covered_counterexample_set_dict: {Not(1 >= y): {frozendict.frozendict({x: 1, y: 2})}, Not(x >= 0): {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, Not(x == y): {frozendict.frozendict({x: 1, y: 2}), frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1}), f

z3_counterexample_input frozendict.frozendict({x: 1, y: 2})


{Not(1 >= y): set(), Not(x >= 0): {frozendict.frozendict({x: -1, y: -2})}, Not(x == y): {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1})}, Not(x >= 1): {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1})}, y <= 1: {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1})}, y >= 0: set(), 1 == x: set(), 1 <= y: set(), x >= 1: set(), 1 <= x: set(), Not(1 == x): {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1})}, Not(y == x): {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1})}, y == 0: set(), Not(x == 0): {frozendict.frozendict({x: -1, y: -2})}, 0 >= y: {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1})}, Not(y >= 0): {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1})}, Not(x == 1): {frozendict.frozendict({x: -1, y: -2}), frozendict.frozendict({x: 0, y: -1})}, 0 >= x: {frozendict.frozendict({x: -1, y: -2}), frozend

In [10]:
print(f'z3_expr_ref {z3_expr_ref}')

z3_counterexample_input = z3_verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    z3_expr_ref = z3_divide_and_conquer_enumerative_approach_instance.send(z3_counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

z3_expr_ref If(y >= x, y, x)
z3_counterexample_input None
